<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0" width=150px> </div>
    <div style="float: left; margin-left: 10px;"> <h1>Generative AI with OpenAI API</h1>
<h1>Code Generation</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint

import pandas as pd
import numpy as np
import sqlite3

import matplotlib
import matplotlib.pyplot as plt 

from ipywidgets import interact
import openai

import os
import gzip

import tqdm as tq
from tqdm.notebook import tqdm

import watermark

%load_ext watermark
%matplotlib inline

We start by printing out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.10.9
IPython version      : 8.10.0

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 22.5.0
Machine     : x86_64
Processor   : i386
CPU cores   : 16
Architecture: 64bit

Git hash: 0ef20a1a126b37fb2a931600722baf12fd1a2389

openai    : 0.28.1
pandas    : 1.5.3
watermark : 2.4.2
sqlite3   : 2.6.0
numpy     : 1.23.5
tqdm      : 4.64.1
matplotlib: 3.7.2



Load default figure style

In [3]:
plt.style.use('d4sci.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Text to Code

In [4]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
messages = [
        {"role": "system", "content": "You are a grumpy but expert Python programmer that interviewing for a job. Please be as concise with your answers a possible."},
        {"role": "user", "content": "Create a recursive Python function to compute Fibonacci numbers. Don't provide any explanation, just the code"},
  ]

In [6]:
response = openai.ChatCompletion.create(
    # GPT-3.5-Turbo now incorporates the Codex functionality
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0,
    max_tokens=1024
)

Which produces the expected result

In [7]:
print(response["choices"][0]['message']["content"])

def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)


Let us define a utility function to make sequential queries easier

In [8]:
def chat(messages, prompt):
    messages.append({"role":"user", "content":prompt})
    
    response = openai.ChatCompletion.create(
        # GPT-3.5-Turbo now incorporates the Codex functionality
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0,
        max_tokens=1024
    )
    
    print(response["choices"][0]['message']['content'])

    messages.append(response["choices"][0]['message'])

# Adding comments

In [9]:
chat(messages, "Can you add comments to this function?")

Sure, here's the code with comments:

```python
def fibonacci(n):
    # Base case: if n is 0 or 1, return n
    if n <= 1:
        return n
    
    # Recursive case: compute the Fibonacci number by summing the previous two numbers
    return fibonacci(n-1) + fibonacci(n-2)
```

Note: The Fibonacci sequence starts with 0 and 1, and each subsequent number is the sum of the previous two numbers.


In [10]:
chat(messages, "What is the purpose of recursion in this piece of code?")

The purpose of recursion in this code is to compute Fibonacci numbers by breaking down the problem into smaller subproblems. The function calls itself with smaller inputs (n-1 and n-2) until it reaches the base case (n <= 1), and then it returns the result. By recursively solving smaller subproblems, the function can compute the Fibonacci number for any given input.


# Explaining Code

In [11]:
code_text = "".join(open("data/EpiModel.py").readlines())

In [12]:
print(code_text)

### −∗− mode : python ; −∗−
# @file EpiModel.py
# @author Bruno Goncalves
######################################################

import networkx as nx
import numpy as np
from numpy import linalg
from numpy import random
import scipy.integrate
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
tqdm.pandas()

class EpiModel(object):
    """Simple Epidemic Model Implementation
    
        Provides a way to implement and numerically integrate 
    """
    def __init__(self, compartments=None):
        self.transitions = nx.MultiDiGraph()
        self.seasonality = None
        
        if compartments is not None:
            self.transitions.add_nodes_from([comp for comp in compartments])
    
    def add_interaction(self, source, target, agent, rate):        
        self.transitions.add_edge(source, target, agent=agent, rate=rate)        
        
    def add_spontaneous(self, source, target, rate):
        self.transitions.add_edge(source, target, rate=rate)



In [13]:
chat(messages, "Please explain what this piece of code does: ```%s```" % code_text)

This code defines a class called `EpiModel` that represents a simple epidemic model. It provides methods to add interactions, spontaneous transitions, and vaccination to the model. The class also includes methods to simulate and numerically integrate the model. The code also includes a main section that demonstrates the usage of the `EpiModel` class by creating an instance of the class, adding interactions and transitions, and then simulating and plotting the results.


In [14]:
chat(messages, "Can you please add a doc string to each function and method?")

Sure, here's the code with added docstrings:

```python
import networkx as nx
import numpy as np
from numpy import linalg
from numpy import random
import scipy.integrate
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
tqdm.pandas()

class EpiModel(object):
    """Simple Epidemic Model Implementation
    
    Provides a way to implement and numerically integrate an epidemic model.
    """

    def __init__(self, compartments=None):
        """Initialize the EpiModel object.
        
        Args:
            compartments (list): List of compartment names.
        """
        self.transitions = nx.MultiDiGraph()
        self.seasonality = None
        
        if compartments is not None:
            self.transitions.add_nodes_from([comp for comp in compartments])
    
    def add_interaction(self, source, target, agent, rate):        
        """Add an interaction to the model.
        
        Args:
            source (str): Source compartment name.
         

# Interacting with a database

Let us open a small test database. This file was downloaded from https://github.com/chineseballer06/Statistical-Analysis-of-Northwind-Database/blob/master/Northwind_small.sqlite

In [15]:
con = sqlite3.connect("data/Northwind_small.sqlite")

In [16]:
messages = [
    {"role": "system", "content": "You're a Database Administrator. Please generate SQL queries to answer the following questions. No comments are necessary."},
    {"role": "user", "content": """
# Table Employee, columns = [Id, LastName, First Name]
# Table Shipper, columns = [Id, CompanyName, Phone]
# Table OrderDetail, columns = [OrderId, ProductId, Quantity]
# Table EmployeeTerritory, columns = [Id, EmployeeId, TerritoryId]
# Table tracks, columns = [TrackId, Name, AlbumId, MediaTypeId, GenreId, Composer, Milliseconds, Bytes, UnitPrice]
    """},
]

In [17]:
chat(messages, "Generate a table with employee first name, last name and territory id")

SELECT e.FirstName, e.LastName, et.TerritoryId
FROM Employee e
JOIN EmployeeTerritory et ON e.Id = et.EmployeeId;


In [18]:
pd.read_sql( """SELECT Employee.FirstName, Employee.LastName, EmployeeTerritory.TerritoryId
FROM Employee
JOIN EmployeeTerritory ON Employee.Id = EmployeeTerritory.EmployeeId;
""", con)

,FirstName,LastName,TerritoryId
0,Nancy,Davolio,06897
1,Nancy,Davolio,19713
2,Andrew,Fuller,01581
3,Andrew,Fuller,01730
4,Andrew,Fuller,01833
5,Andrew,Fuller,02116
6,Andrew,Fuller,02139
7,Andrew,Fuller,02184
8,Andrew,Fuller,40222
9,Janet,Leverling,30346


<center>
     <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>